In [9]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import optuna


In [10]:

data = pd.read_csv('data/stroke_woe_smote.csv')

In [11]:
# Convertir columnas booleanas a enteros (0 y 1)
bool_columns = data.select_dtypes(include=['bool']).columns
data[bool_columns] = data[bool_columns].astype(int)

# Comprobar y convertir a numérico
data = data.apply(pd.to_numeric, errors='coerce')
data = data.dropna()  # Eliminar filas con NaN

In [12]:
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values
y = y.flatten()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
def objective(trial):
    n_layers = trial.suggest_int('n_layers', 3, 7)  # Número de capas ocultas
    n_units = trial.suggest_int('n_units', 256, 1024)   # Número de neuronas por capa
    activation = trial.suggest_categorical('activation', ['relu'])  # Funciones de activación
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)  # Tasa de dropout
    batch_size = trial.suggest_int('batch_size', 32, 512)  # Tamaño del lote
    epochs = trial.suggest_int('epochs', 100, 300)  # Número de épocas

    # Crear el modelo
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=(X_train.shape[1],)))

    for _ in range(n_layers):
        model.add(keras.layers.Dense(n_units, activation=activation))
        model.add(keras.layers.Dropout(dropout_rate))

    model.add(keras.layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

    accuracy = model.evaluate(X_test, y_test, verbose=0)[1]
    return accuracy


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print("Mejores hiperparámetros: ", study.best_params)
print("Mejor precisión: ", study.best_value)